# Geração de Texto usando LCEL e Runnables com Llama 2

####**📝 Descrição**  

Este projeto implementa um pipeline de geração de texto utilizando **LangChain Expression Language (LCEL)** e **Runnables** com o modelo **Llama 2** da Meta. Através dessa abordagem, é possível criar **prompts estruturados**, otimizar a execução do modelo e adicionar funções personalizadas, como a contagem de palavras no texto gerado.  

A quantização via **BitsAndBytesConfig** permite que o modelo seja carregado em **4-bit**, reduzindo o consumo de memória e tornando possível sua execução em ambientes limitados, como o Google Colab.  

####**📊 Ficha Técnica**

| 🔍 **Item** | 📄 **Descrição** |
|---|---|
| **🛠️ Tecnologias** | Python, LangChain, Transformers, PyTorch, BitsAndBytes |
| **📦 Dependências** | `transformers`, `torch`, `bitsandbytes`, `accelerate`, `langchain-community` |
| **⚙️ Funcionalidade** | Geração de texto otimizada com LCEL e Runnables |
| **📌 Modelo Utilizado** | `meta-llama/Llama-2-7b-chat-hf` |
| **📉 Quantização** | 4-bit (`BitsAndBytesConfig`) |
| **🔑 Autenticação** | Token da Hugging Face (`getpass`) |
| **📜 Estrutura de Prompt** | `ChatPromptTemplate` com mensagens do sistema e do usuário |
| **🔧 Funções Customizadas** | Contagem de palavras no texto gerado (`RunnableLambda`) |

## instalações e configurações

In [12]:
!pip install -U bitsandbytes # versão atualizada necessaria
!pip install accelerate
!pip install langchain-community

from langchain_core.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda
from langchain_community.llms import HuggingFacePipeline
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, pipeline
import torch
import getpass
#import os

ERROR: Operation cancelled by user
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/cli/base_command.py", line 179, in exc_logging_wrapper
    status = run_func(*args)
             ^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/cli/req_command.py", line 67, in wrapper
    return func(self, options, args)
           ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/commands/install.py", line 447, in run
    conflicts = self._determine_conflicts(to_install)
                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/commands/install.py", line 578, in _determine_conflicts
    return check_install_conflicts(to_install)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/operations/check.py", line 101, in check_install_conflicts
    package_set, _ = create_package_

In [4]:
# Solicita o token da Hugging Face e armazena na variável de ambiente
os.environ["HF_TOKEN"] = getpass.getpass()

··········


## configurando o modelo

In [5]:
# Escolha do modelo
model_name = "meta-llama/Llama-2-7b-chat-hf"

# Configuração para carregar o modelo em 4-bit (economiza memória)
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,  # Alternativamente, use load_in_8bit=True para 8-bit
    bnb_4bit_compute_dtype=torch.float16,  # Define precisão do cálculo
    bnb_4bit_use_double_quant=True,  # Usa quantização dupla para eficiência
    bnb_4bit_quant_type="nf4"  # Quantização NF4 (melhor que int8)
)

# Carregar tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Carregar modelo com quantização
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=quantization_config,
    device_map="auto"  # Usa GPU automaticamente, se disponível
)

tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

## criando o pipeline para o modelo


In [8]:
# Criando o pipeline para o modelo
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=200)
llm = HuggingFacePipeline(pipeline=pipe)

Device set to use cuda:0


## criando o prompt estruturado para chat

In [9]:
# Criando o prompt estruturado para chat
system_message = SystemMessagePromptTemplate.from_template(
    "Você é um assistente e está respondendo perguntas gerais."
    )
user_message = HumanMessagePromptTemplate.from_template(
    "Explique para mim brevemente o conceito de {topic}, de forma clara e objetiva. Escreva em no máximo {tamanho}."
    )

# Criando o ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages([system_message, user_message])

## função customizada com runnables

In [10]:
# Função para contar palavras
count = RunnableLambda(lambda x: f"Palavras: {len(x.split())}\n{x}")

## execução

In [11]:
# Definição da cadeia de execução
chain = prompt | llm | StrOutputParser() | count

# Execução da cadeia
output = chain.invoke({"topic": "arqueologia", "tamanho": "1 frase"})
print(output)

Palavras: 61
System: Você é um assistente e está respondendo perguntas gerais.
Human: Explique para mim brevemente o conceito de arqueologia, de forma clara e objetiva. Escreva em no máximo 1 frase.
Assistente: A arqueologia é o estudo científico dos restos materiais e culturais do passado, com o objetivo de entender a história e a cultura de diferentes sociedades humanas através do tempo.
